# Experimentation notebook (testing in this project)

Sources:
- https://www.youtube.com/watch?v=BrsocJb-fAo
- https://github.com/svpino/youtube-rag

In [1]:
# Imports
import os
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser



In [207]:
# Model setup
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
parser = StrOutputParser()

In [208]:
# Template setup

template = """
Respondé las preguntas basandote en el siguiente contexto. Si no podes responder una pregunta. 
Responde: "Necesito más información para responder esta pregunta."

Contexto: {context}

Pregunta: {question}

"""


In [209]:
# Import file 

with open('resumen_tesis.txt', 'r', encoding='utf-8') as file:
    texto = file.read()
texto 

'UNIVERSIDAD DE BUENOS AIRES\nFacultad de Ciencias Exactas y Naturales\nDepartamento de Física\nModelado y calibración en dinámica peatonal:\npropuestas para mejorar las evacuaciones\nen estado de pánico\nTesis presentada para optar al título de Doctor de la Universidad de\nBuenos Aires en el área Ciencias Físicas\nLic. Ignacio Mariano Sticco\nDirector de tesis: Dr. Guillermo Alberto Frank\nConsejero de Estudios: Dr. Ariel Chernomoretz\nLugar de trabajo: Instituto de Física de Buenos Aires (IFIBA)\nBuenos Aires, 2023\nResumen\nEsta tesis estudia la dinámica de multitudes en estado de pánico. Para describir el comportamiento de dichas multitudes se utiliza el modelo de fuerza social de Helbing. En este modelo los individuos se representan como discos blandos que están sometidos a distintos tipos de fuerzas, y la evolución temporal del sistema queda determinada por las ecuaciones de movimiento de la mecánica clásica. La ventaja de este modelo es que permite estudiar fenómenos macroscópic

In [210]:
# Invoke
context = texto

pregunta = "De que trata la tesis?"
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | model | parser
chain.invoke({
    "context": context,
    "question": pregunta
})

'La tesis trata sobre el estudio de la dinámica de multitudes en estado de pánico, utilizando el modelo de fuerza social de Helbing para describir el comportamiento de las multitudes en situaciones de evacuación de emergencia. También se exploran mejoras estructurales para las evacuaciones de emergencia, basadas en la calibración de parámetros y en el análisis de recintos con vestíbulos.'

### Pruebo embedear un texto que yo le paso

In [211]:
# Load and split the document
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

#loader = PyPDFLoader("tesis_doctoral.pdf")
loader = TextLoader("tesis_doctoral.txt", encoding='utf-8')

text_documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(text_documents)

In [212]:
# Embed the chunked documents
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch
embeddings = OpenAIEmbeddings()
vectorstore = DocArrayInMemorySearch.from_documents(documents, embeddings)

The retriever will run a similarity search in the vector store and return the most similar chunked documents back to the next step in the chain.

In [213]:
# Setup, create and invoke the chain with retriver. 
from langchain_core.runnables import RunnableParallel, RunnablePassthrough


setup = RunnableParallel(context=vectorstore.as_retriever(), question=RunnablePassthrough())

chain = (
    setup   
    | prompt
    | model
    | parser
)
chain.invoke("Dame la formula matematica de los blocking cluster")

'Cg:Piϵ Cg⇔ ∃j ϵ C g/rij<(Ri+Rj) (2.10)'

In [ ]:
# Preguntas que responde bien con chunk_size=1000, chunk_overlap=100 y arrivo txt (no pdf)

"Quien es el autor de la tesis?"

"Quien es el director de la tesis?"

"Cuales son los valores de kn y kt que mejor reproducen los datos empiricos?"

"Resumime el capitulo 3"

"Cual es el porcentaje de aumento del flujo de evacuacion por agregar un vestibulo de 2 puertas?"

"Que es el diagrama fundamental?"

"Dame la formula matematica de la fuerza social y explicame cada termino"

"Dame la formula matematica de los blocking cluster"

"Que son los blocking clusters?"

"Que produce un mayor flujo de evacuacion, un vestibulo de 1 puerta o 2 puertas? y por que?"
(chunk_size=10000, chunk_overlap=1000)

# Preguntas que aun no responde
"Describime la Figura 4.2"

"Cual es el autor mas citado en esta tesis?"

"En que año se escribio esta tesis?"

"Cuales son las fuerzas del modelo de fuerza mas relevantes en situaciones de alta densidad?"

"Mencioname alguna referencia bibliografica de la tesis"

"Cuales son las conclusiones de la tesis?"

# Preguntas que responde mal

"Que es el panico en el contexto de las evacuaciones de emergencia?"

"Cual es la distancia a la puerta de salida y el panel para producir un flujo de evacuacion optimo?"